[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/espnet/notebook/blob/master/espnet2_tts_realtime_demo.ipynb)

# ESPnet2-TTS realtime demonstration

This notebook provides a demonstration of the realtime E2E-TTS using ESPnet2-TTS and ParallelWaveGAN (+ MelGAN).

- ESPnet2-TTS: https://github.com/espnet/espnet/tree/master/egs2/TEMPLATE/tts1
- ParallelWaveGAN: https://github.com/kan-bayashi/ParallelWaveGAN

Author: Tomoki Hayashi ([@kan-bayashi](https://github.com/kan-bayashi))

## Installation

In [ ]:
# NOTE: pip shows imcompatible errors due to preinstalled libraries but you do not need to care
!pip install -q espnet==0.9.5 parallel_wavegan==0.4.8

## Single speaker model demo

### Model Selection

Please select models by comment out.

English, Japanese, and Mandarin are supported.

You can try Tacotron2, FastSpeech, and FastSpeech2 as the text2mel model.  
And you can use Parallel WaveGAN and Multi-band MelGAN as the vocoder model.

In [ ]:
###################################
#          ENGLISH MODELS         #
###################################
fs, lang = 22050, "English"
# tag = "kan-bayashi/ljspeech_tacotron2"
# tag = "kan-bayashi/ljspeech_fastspeech"
tag_dir= "/home/airenas/projects/espnet/egs2/ljspeech/tts1/exp/tts_train_fastspeech2_raw_phn_tacotron_g2p_en_no_space/"
tag = "20epoch.pth"
# tag = "kan-bayashi/ljspeech_conformer_fastspeech2"
vocoder_tag = "ljspeech_parallel_wavegan.v1"
# vocoder_tag = "ljspeech_full_band_melgan.v2"
# vocoder_tag = "ljspeech_multi_band_melgan.v2"


### Model Setup

In [ ]:
import time
import torch
from espnet2.bin.tts_inference import Text2Speech
from parallel_wavegan.utils import download_pretrained_model
from parallel_wavegan.utils import load_model
mf = {'train_config': tag_dir + 'config-run.yaml',
 'model_file': tag_dir + tag}
dev="cuda"
text2speech = Text2Speech(
    **mf,
    device=dev,
    # Only for Tacotron 2
    threshold=0.5,
    minlenratio=0.0,
    maxlenratio=10.0,
    use_att_constraint=False,
    backward_window=1,
    forward_window=3,
    # Only for FastSpeech & FastSpeech2
    speed_control_alpha=1.0,
)
text2speech.spc2wav = None  # Disable griffin-lim
# NOTE: Sometimes download is failed due to "Permission denied". That is 
#   the limitation of google drive. Please retry after serveral hours.
vocoder = load_model(download_pretrained_model(vocoder_tag)).to(dev).eval()
vocoder.remove_weight_norm()

### Synthesis

In [ ]:
# decide the input sentence by yourself
x = "Hi vocoder. How are you?"

# synthesis
with torch.no_grad():
    start = time.time()
    wav, c, *_ = text2speech(x)
    wav = vocoder.inference(c)
rtf = (time.time() - start) / (len(wav) / fs)
print(f"RTF = {rtf:5f}")

# let us listen to generated samples
from IPython.display import display, Audio
display(Audio(wav.view(-1).cpu().numpy(), rate=fs))

## Multi-speaker Model Demo

### Model Selection

Please select models by comment out.

Now we provide only English multi-speaker pretrained model.

In [ ]:
###################################
#          ENGLISH MODELS         #
###################################
fs, lang = 24000, "English"
tag = "kan-bayashi/vctk_gst_tacotron2"
# tag = "kan-bayashi/vctk_gst_transformer"
vocoder_tag = "vctk_parallel_wavegan.v1"
# vocoder_tag = "vctk_multi_band_melgan.v2"

### Model Setup

In [ ]:
import time
import torch
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.tts_inference import Text2Speech
from parallel_wavegan.utils import download_pretrained_model
from parallel_wavegan.utils import load_model
d = ModelDownloader()
text2speech = Text2Speech(
    **d.download_and_unpack(tag),
    device="cuda",
    # Only for Tacotron 2
    threshold=0.5,
    minlenratio=0.0,
    maxlenratio=10.0,
    use_att_constraint=False,
    backward_window=1,
    forward_window=3,
    # Only for FastSpeech & FastSpeech2
    speed_control_alpha=1.0,
)
text2speech.spc2wav = None  # Disable griffin-lim
# NOTE: Sometimes download is failed due to "Permission denied". That is 
#   the limitation of google drive. Please retry after serveral hours.
vocoder = load_model(download_pretrained_model(vocoder_tag)).to("cuda").eval()
vocoder.remove_weight_norm()

### Synthesis

For multi-speaker model, we need to provide the reference speech to decide the speaker characteristics.  
You can use any speech but please make sure the sampling rate is matched.

In [ ]:
# decide the input sentence by yourself
print(f"Input your favorite sentence in {lang}.")
x = input()

# you can change here to load your own reference speech
# e.g.
# import soundfile as sf
# speech, fs = sf.read("/path/to/reference.wav")
# speech = torch.from_numpy(speech).float()
speech = torch.randn(50000,)

# synthesis
with torch.no_grad():
    start = time.time()
    wav, c, *_ = text2speech(x, speech=speech)
    wav = vocoder.inference(c)
rtf = (time.time() - start) / (len(wav) / fs)
print(f"RTF = {rtf:5f}")

# let us listen to generated samples
from IPython.display import display, Audio
display(Audio(wav.view(-1).cpu().numpy(), rate=fs))